In [36]:
import pathlib
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipctk
from tqdm.notebook import tqdm

In [37]:
from paper_style import *

In [38]:
root = pathlib.Path("/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d")

x-axis: timestep for spike ball nrefs + ours

y-axis: total contacts, total IP energy, max stress, # dofs

In [39]:
energy_df = pd.read_csv(root / "restart_031" / "2023_04_13_21_55_40_703" / "energy.csv")
stats_df =  pd.read_csv(root / "restart_031" / "2023_04_13_21_55_40_703" / "stats.csv")
contact_df =  pd.read_csv(root / "restart_031" / "2023_04_13_21_55_40_703" / "contacts.csv")

In [40]:
energy_df

,i,elastic_energy,body_energy,inertia,contact_form,AL_energy,total_energy
0,1,0.000001,-0.526906,0.000499,0.000000,0,-0.526406
1,2,0.000001,-0.526906,0.000499,0.000000,0,-0.526406
2,3,0.000001,-0.526906,0.000499,0.000000,0,-0.526406
3,4,0.000876,-0.559775,0.000662,0.000010,0,-0.557878
4,5,0.000545,-0.559723,0.000515,0.000005,0,-0.558474
...,...,...,...,...,...,...,...
140,141,0.183408,-1.158724,0.001337,0.000341,0,-0.971297
141,142,0.162810,-1.138026,0.000997,0.000293,0,-0.972117
142,143,0.161645,-1.137940,0.001015,0.000283,0,-0.972953
143,144,0.161627,-1.138000,0.000999,0.000291,0,-0.972942


In [74]:
import subprocess

csv_root = pathlib.Path("/Users/zachary/Downloads/remeshing-csv/")
for log in tqdm(list(csv_root.glob("**/log.txt"))):
    subprocess.run([
        "grep", "-E", "inverting|niters", str(log),
    ], stdout=open(log.parent / "solver_times.txt", "w"))

  0%|          | 0/25 [00:00<?, ?it/s]

In [77]:
for log in tqdm(list(csv_root.glob("**/solver_times.txt"))):
    solve_times = {"iterations": []}
    with open(log) as f:
        lines = f.readlines()
    for line in lines:
        if "inverting" not in line:
            solve_times["iterations"].append(int(line.split("niters=")[1].split()[0]))
            continue
        line = line.split()[5:]
        for i in range(len(line) // 2):
            if line[2*i] not in solve_times:
                solve_times[line[2*i]] = []
            solve_times[line[2*i]].append(float(line[2*i+1].strip(",").strip("s")))
    pd.DataFrame(solve_times).to_csv(log.with_suffix(".csv"))

  0%|          | 0/25 [00:00<?, ?it/s]

In [86]:
df = pd.DataFrame(index=["spikes3d", "masticator", "rollers", "twisting-beam", "ball-wall"], columns=["UR 0", "UR 1", "UR 2", "UR 3", "Ours"])  
for csv in tqdm(list(csv_root.glob("**/solver_times.csv"))):
    solver_df = pd.read_csv(csv)
    idx = csv.parents[2].name
    col = csv.parents[1].name
    if "nref" in col: 
        col = "UR " + col.split("nref")[-1]
    else:
        col = "Ours"
    df.loc[idx][col] = float((solver_df["inverting"] / solver_df["iterations"]).mean())
name_map = {
    "spikes3d": "Ball on spikes (\cref{fig:teaser}) ",
    "masticator": "Masticator (\cref{fig:masticator-remeshing-demo})",
    "rollers": "Gorilla rollers (\cref{fig:gorilla-rollers})",
    "twisting-beam": "Bar-twist (\cref{fig:bar-remeshing-demo})",
    "ball-wall": "Impacting ball (\cref{fig:sphere-remeshing-demo})",
}
tab = df.to_latex(float_format="{:.2f}".format)
for name, pretty_name in name_map.items():
    tab = tab.replace(name, pretty_name)
print(tab)

  0%|          | 0/25 [00:00<?, ?it/s]

\begin{tabular}{llllll}
\toprule
{} & UR 0 & UR 1 &  UR 2 &   UR 3 & Ours \\
\midrule
Ball on spikes (\cref{fig:teaser})       & 0.08 & 0.81 &  9.88 & 201.77 & 0.25 \\
Masticator (\cref{fig:masticator-remeshing-demo})    & 0.01 & 0.06 &  0.78 &   9.39 & 0.22 \\
Gorilla rollers (\cref{fig:gorilla-rollers})       & 0.12 & 1.23 & 16.82 & 245.44 & 0.17 \\
Bar-twist (\cref{fig:bar-remeshing-demo}) & 0.00 & 0.05 &  0.57 &   8.77 & 0.21 \\
Impacting ball (\cref{fig:sphere-remeshing-demo})     & 0.02 & 0.28 &  3.60 & 126.68 & 0.37 \\
\bottomrule
\end{tabular}



/var/folders/y1/l7pg454j0jv6b028zqg1frsc0000gn/T/ipykernel_48161/2188042587.py:18: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [43]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
x = 0.01 * np.arange(48) + 0.31
dirs = sorted(list(root.glob("*noremesh*/2023*"))) + [root / "restart_031" / "2023_04_13_21_55_40_703"]
for i, dir in enumerate(dirs):
    print(dir)
    energy_df = pd.read_csv(dir / "energy.csv")
    stats_df =  pd.read_csv(dir / "stats.csv")
    contact_df =  pd.read_csv(dir / "contacts.csv")
    solver_times =  pd.read_csv(dir / "solve_times.csv")
    
    stride = 1 if "noremesh" in dir.parent.name else 3
    name = str(dir.parent.name).split("-")[-1] if stride == 1 else "Ours"
    if stride == 1:
        name = name.replace("nref", "UR")
        name = name.replace("0", " 0").replace("1", " 1").replace("2", " 2").replace("3", " 3")
    # if name == "UR 3":
    #     continue

    ys = []
    offset = 1
    # if "noremesh" not in dir.parent.name:
    #     ys.append(np.array(stats_df["#V"])[:x.size] * 3)
    #     offset = 0
    ys.extend([
        # np.array(contact_df["# Contacts"])[:x.size], # num contacts
        # np.array(energy_df["total_energy"])[::stride][:x.size],
        # np.full(x.shape, np.nan), # max stress
        # np.array(energy_df["elastic_energy"])[::stride][:x.size],
        np.array(stats_df["#V"])[:x.size] * 3,
        np.array(solver_times["inverting"])[:x.size],
        np.array(stats_df["forward"] + stats_df["remeshing"] + stats_df["global_relaxation"])[:x.size],
        # np.array(stats_df["peak_mem"])[:x.size],
    ])
    for j, y in enumerate(ys):
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name=name, showlegend=(j==0), line_color=matlab_colors[i]), row=j+1, col=1)
    
titles = {
    f"yaxis{i+1}": dict(
        title=title,
        type="log",
    )
    for i, title in enumerate([
        # "# Contacts",
        # "Energy (J)",
        # "Elastic Energy (J)",
        "# DOF",
        "Avg. Linear<br>Solve Time (s)",
        "Running<br>Time (s)",
        # "Peak Memory<br>(GiB)",
    ])
}
titles[f"xaxis{len(titles)}_title"] = "Time (s)"

fig.update_layout(
    **titles,
    **paper_style(aspect_ratio=0.75)
)
fig.show()
fig.write_image("nref-plot.pdf")

/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d/restart_031-noremesh-nref0/2023_04_13_21_55_40_484
/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d/restart_031-noremesh-nref1/2023_04_13_21_55_40_512
/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d/restart_031-noremesh-nref2/2023_04_13_21_55_40_525
/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d/restart_031-noremesh-nref3/2023_04_19_13_40_13_148
/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-csv/spikes3d/restart_031/2023_04_13_21_55_40_703


In [44]:
# import meshio
# from tqdm import tqdm


# ply_dir = pathlib.Path("/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/remeshing-plys/spikes3d")
# for dir in ply_dir.glob("*/*"):
#     if dir.name == "old":
#         continue
#     # if (root / dir.relative_to(ply_dir) / "contacts.csv").exists():
#     #     continue
#     print(dir)
#     contacts = []
#     for i in tqdm(range(min(48, len(list(dir.glob("*.ply")))))):
#         ply = dir / f"{i:04d}.ply"
#         mesh = meshio.read(ply)
#         V = mesh.points
#         F = mesh.cells[0].data
#         cmesh = ipctk.CollisionMesh(V, ipctk.edges(F), F)
#         constraints = ipctk.Constraints()
#         constraints.build(cmesh, V, 1e-3)
#         contacts.append(len(constraints))
#     pd.DataFrame({"# Contacts": contacts}).to_csv(root / dir.relative_to(ply_dir) / "contacts.csv", index=False)

